In [68]:
from quandl import get
from sklearn import preprocessing, svm
from sklearn.model_selection import train_test_split as ttsplit
from sklearn.linear_model import LinearRegression as lr
from matplotlib import pyplot, style
from pandas import DataFrame
import numpy
from pickle import dump, load

In [69]:
style.use("ggplot")

In [70]:
key = open("../key").read()[:-1]
df = get("XFRA/SES", authtoken = key)

In [71]:
def label_detect(open_v, close_v):
    if close_v > open_v:
        return 1
    else:
        return 0

In [72]:
df["label"] = list(map(label_detect, df["Open"], df["Close"]))

In [73]:
df = df.fillna(0)
x = numpy.array(df.drop(["label"], 1))
y = numpy.array(df["label"])
x = preprocessing.scale(x)

In [74]:
len_x = len(x)
br_point = int((len_x / 100) * 70)
x_now = x[:br_point]
x_fut = x[br_point:]
y_now = y[:br_point]
y_fut = y[br_point:]
x_train, x_test, y_train, y_test = ttsplit(x_now, y_now, test_size = 0.2)

In [75]:
buy_signal_lr = lr(n_jobs = 10)
buy_signal_lr.fit(x_train, y_train)
buy_signal_lr.score(x_test, y_test)

0.44337880469445257

In [76]:
with open("trainnedmodel_lrpickle","wb") as f: dump(buy_signal_lr, f)

In [77]:
buy_signal_lr = load(open("trainnedmodel_lrpickle", "rb"))

In [78]:
buy_signal_svm = svm.SVR(gamma = "scale")
buy_signal_svm.fit(x_train, y_train)
buy_signal_svm.score(x_test, y_test)

0.1706061109913849

In [84]:
forecast_set_lr = buy_signal_lr.predict(numpy.array(x_fut["Close"]))
# forecast_set_svm = buy_signal_svm.predict(x_fut["Close"])

IndexError: only integers, slices (`:`), ellipsis (`...`), numpy.newaxis (`None`) and integer or boolean arrays are valid indices

In [ ]:
nan_row = [[numpy.nan] * 7] * len_x
df_realsig = DataFrame(x_now.tolist() + nan_row[br_point:], columns = list(df.columns)[:-1], index = df.index)
df_futsig = DataFrame(nan_row[:br_point] + x_fut.tolist(), columns = list(df.columns)[:-1], index = df.index)

In [ ]:
df_realsig["Close"].plot()
df_futsig["Close"].plot()
pyplot.xlabel("Date")
pyplot.ylabel("Price")
pyplot.show()

In [ ]:
df["Open"].plot.kde()
pyplot.show()

In [ ]:
df["Volume"].plot.kde()
pyplot.show()